In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [4]:
BASE_DIR = 'data_raw'

# Port Distances Dataset

In [6]:
df = pd.read_csv(f'{BASE_DIR}/Port Distances.csv')
df.head(2)

,PORT_NAME_FROM,PORT_NAME_TO,DISTANCE
0,AAHEIM,HALDIA,8652.67
1,AAHEIM,PARANAGUA,6304.48


In [10]:
print(f"shape: {df.shape}")
print(f"dtypes:\n{df.dtypes}")

shape: (15533, 3)
dtypes:
PORT_NAME_FROM        str
PORT_NAME_TO          str
DISTANCE          float64
dtype: object


In [11]:
# check for null values
print(f"null values:\n{df.isna().sum()}")

# summary statistics for `DISTANCE`
print(df["DISTANCE"].describe())
print(f"non-positive distances: {(df["DISTANCE"] <= 0).sum()}")

null values:
PORT_NAME_FROM    0
PORT_NAME_TO      0
DISTANCE          0
dtype: int64
count    15533.000000
mean      5067.643422
std       3287.136255
min          6.860000
25%       2370.900000
50%       4545.420000
75%       7293.000000
max      13558.480000
Name: DISTANCE, dtype: float64
non-positive distances: 0


In [20]:
from_ports = set(df["PORT_NAME_FROM"].unique())
to_ports = set(df["PORT_NAME_TO"].unique())
all_ports = from_ports | to_ports

print(f"unique FROM_PORTs: {len(from_ports)}")
print(f"unique TO_PORTs: {len(to_ports)}")
print(f"total unique ports: {len(all_ports)}")

unique FROM_PORTs: 1279
unique TO_PORTs: 1432
total unique ports: 1950


In [21]:
# if port appears only as TO or only as FROM, it indicates incomplete data
only_to_ports = to_ports - from_ports
only_from_ports = from_ports - to_ports


print(f"ports only appearing as TO (no outgoing): {len(sorted(list(only_to_ports)))}")
print(f"ports only appearing as FROM (no incoming): {len(sorted(list(only_from_ports)))}")

ports only appearing as TO (no outgoing): 671
ports only appearing as FROM (no incoming): 518


In [23]:
import plotly.express as px

coverage = pd.DataFrame({
    "category": ["ports_total", "only_as_to (no outgoing)", "only_as_from (no incoming)"],
    "count": [len(all_ports), len(only_to_ports), len(only_from_ports)]
})
px.bar(coverage, x="category", y="count", title="Port coverage gaps")


In [27]:
# Check for directedness of distances
pairs = set(zip(df["PORT_NAME_FROM"], df["PORT_NAME_TO"]))
rev_pairs = set((b,a) for (a,b) in pairs)

reverse_coverage = len(pairs & rev_pairs) / len(pairs)
print("Reverse-pair availability:", reverse_coverage)

# quantify differences in distances for reverse pairs
dist_map = {(a,b): d for a,b,d in df[["PORT_NAME_FROM","PORT_NAME_TO","DISTANCE"]].itertuples(index=False)}
sym = []
for (a,b), d in dist_map.items():
    if (b,a) in dist_map:
        sym.append((a,b,d,dist_map[(b,a)], abs(d-dist_map[(b,a)])))

sym_df = pd.DataFrame(sym, columns=["from","to","d_ab","d_ba","abs_diff"])
sym_df["pct_diff_vs_mean"] = sym_df["abs_diff"] / ((sym_df["d_ab"]+sym_df["d_ba"])/2)

print(f"Pairs with reverse present: {len(sym_df)}")
print(f"Share mismatched (abs_diff>0): {(sym_df["abs_diff"]>0).mean()}")
sym_df[["abs_diff","pct_diff_vs_mean"]].describe()

Reverse-pair availability: 0.019313719178523144
Pairs with reverse present: 300
Share mismatched (abs_diff>0): 0.9666666666666667


,abs_diff,pct_diff_vs_mean
count,300.000000,300.000000
mean,188.474800,0.038549
std,195.444543,0.025624
min,0.000000,0.000000
25%,32.190000,0.019787
50%,116.930000,0.034283
75%,311.880000,0.058647
max,1017.100000,0.146341


In [28]:
directed_summary = pd.DataFrame({
    "category": ["Pairs with reverse present", "Pairs without reverse"],
    "count": [len(pairs & rev_pairs), len(pairs) - len(pairs & rev_pairs)]
})
px.bar(directed_summary, x="category", y="count", title="Directedness check: reverse-pair availability")

px.histogram(sym_df, x="pct_diff_vs_mean", nbins=40,
             title="If reverse exists: abs % distance difference vs mean (directed routing effect)")
